In [ ]:
import numpy as np
import pandas as pd
from IPython.display import display
from keras.preprocessing.text import Tokenizer
from keras.preprocessing.sequence import pad_sequences
from keras.models import Sequential
from keras.layers import Embedding, Conv1D, MaxPooling1D, LSTM, Dense, Dropout
from keras.layers import TextVectorization
from sklearn.metrics import accuracy_score
import tensorflow as tf
import keras
import nltk
from nltk.stem import 	WordNetLemmatizer
from nltk.stem.porter import PorterStemmer
from nltk.corpus import stopwords
import preprocessor as p
import re

In [ ]:
# nltk.download('punkt')
# nltk.download('wordnet')
# nltk.download('stopwords')
stop_words = set(stopwords.words('english'))

In [ ]:
# !wget http://nlp.stanford.edu/data/glove.6B.zip
# !unzip -q glove.6B.zip

In [ ]:
%cd 550\ Final\ Project

[Errno 2] No such file or directory: '550 Final Project'
/content/550 Final Project


Load the datasets

In [ ]:
train = pd.read_csv("data/Constraint_Train.csv")
val = pd.read_csv("data/Constraint_Val.csv")
test = pd.read_csv("data/english_test_with_labels.csv")
train_c, train_l = train['tweet'].to_numpy(), train['label'].to_numpy()
train_l = np.array([0 if i == 'fake' else 1 for i in train_l])
val_c, val_l = val['tweet'].to_numpy(), val['label'].to_numpy()
val_l = np.array([0 if i == 'fake' else 1 for i in val_l])
test_c, test_l = test['tweet'].to_numpy(), test['label'].to_numpy()
test_l = np.array([0 if i == 'fake' else 1 for i in test_l])
# print(train_c)
# display(train.head())

Preprocess

---

In [ ]:
wordnet_lemmatizer = WordNetLemmatizer()
porter_stemmer  = PorterStemmer()

p.set_options(p.OPT.URL, p.OPT.EMOJI)

def preprocess(row, lemmatizer, stemmer):
    txt = row
    txt = p.clean(txt)
    tokenization = nltk.word_tokenize(txt)     
    tokenization = [w for w in tokenization if not w in stop_words]
    # txt = ' '.join([porter_stemmer.stem(w) for w in tokenization])
    # txt = ' '.join([lemmatizer.lemmatize(w) for w in txt])
    txt = re.sub(r'[^a-zA-Z ]', '', txt).lower().strip()    
    return txt

train_c = [preprocess(x, wordnet_lemmatizer, porter_stemmer) for x in train_c]
val_c = [preprocess(x, wordnet_lemmatizer, porter_stemmer) for x in val_c]
test_c = [preprocess(x, wordnet_lemmatizer, porter_stemmer) for x in test_c]



---

In [ ]:
# the model will remember only the top 20000 most common words
max_words = 20000
max_len = 300
voc = np.array(train_c + val_c + test_c)
token = Tokenizer(num_words=max_words, lower=True, split=' ')
token.fit_on_texts(voc)
sequences = token.texts_to_sequences(train_c)
train_sequences_padded = pad_sequences(sequences, maxlen=max_len)

sequences = token.texts_to_sequences(val_c)
val_sequences_padded = pad_sequences(sequences, maxlen=max_len)

sequences = token.texts_to_sequences(test_c)
test_sequences_padded = pad_sequences(sequences, maxlen=max_len)

In [ ]:
def net():
    model = Sequential([
        Embedding(max_words, 100, input_length=300),

        Conv1D(32, 8, activation='relu', padding="same"),
        MaxPooling1D(2),

        LSTM(32),
        Dense(10, activation="relu"),
        Dropout(0.5),
        Dense(1, activation="sigmoid")
    ])
    opt = tf.keras.optimizers.Adam(learning_rate=0.0005)
    model.compile(loss=tf.keras.losses.BinaryCrossentropy(),
                  optimizer=opt, metrics=['accuracy'])
    return model

In [ ]:
model = net()
model.fit(train_sequences_padded, train_l, batch_size=32, epochs=5,
          validation_data=(val_sequences_padded, val_l))

Epoch 1/5
201/201 [==============================] - 5s 14ms/step - loss: 0.5027 - accuracy: 0.7671 - val_loss: 0.2807 - val_accuracy: 0.8953
Epoch 2/5
201/201 [==============================] - 2s 11ms/step - loss: 0.2284 - accuracy: 0.9198 - val_loss: 0.2172 - val_accuracy: 0.9178
Epoch 3/5
201/201 [==============================] - 2s 11ms/step - loss: 0.1204 - accuracy: 0.9625 - val_loss: 0.2192 - val_accuracy: 0.9215
Epoch 4/5
201/201 [==============================] - 2s 12ms/step - loss: 0.0717 - accuracy: 0.9808 - val_loss: 0.2588 - val_accuracy: 0.9243
Epoch 5/5
201/201 [==============================] - 2s 12ms/step - loss: 0.0496 - accuracy: 0.9829 - val_loss: 0.3309 - val_accuracy: 0.9262


In [ ]:
pred = model.predict(test_sequences_padded)
pred = pred.reshape(2140)
pred = np.array([0 if i < 0.5 else 1 for i in pred])

In [ ]:
acc = accuracy_score(test_l, pred)
print(acc)

0.935981308411215
